In [1]:
import pandas as pd

### Initialize graphdatascience client

In [2]:
# pip install graphdatascience
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to your setup
# NEO4J_URI could look similar to "bolt://my-server.neo4j.io:7687"
gds = GraphDataScience("bolt://localhost:7687", auth=("neo4j", "0306"))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

2.2.5


### Search for names in list

The names parser assumes that the last name is either at the end or if the name contains a "," at the beginning. It can deal with lower/upper case, different spellings for german letters, combined names ("-"), several first names. It can not del with additional "," other special titles like "zu" or "von" as aprt of the last name (will assume they are part of the first name)

In [32]:
df = pd.read_csv('names.csv', sep=";")
df.head()

In [34]:
def names_lists(name):    
    last_name = []
    first_name = []

    # remove "Prof." & "Dr." & ","
    # lower case everywhere
    # split at space;
    split_name = name.replace("Prof.", "").replace("Dr.", "").replace(",", "").lower().split()


    # split first and last name & switch positions if ","
    if name.count(",") > 0:
        last_name.append(split_name[0]) #start:stop:step
        first_name = split_name[1:]
    else:
        last_name.append(split_name[len(split_name)-1])
        first_name = split_name[:len(split_name)-1]


    # replace ß, ü, ä, ö
    def extending_german_letters(names):
        ret_names = []
        for name in names:
            ret_names.append(name)
            count = name.count("ä")
            if count > 0:
                ret_names.append(name.replace("ä", "ae"))
                ret_names.append(name.replace("ä", "a"))
            count = name.count("ö")
            if count > 0:
                ret_names.append(name.replace("ö", "oe"))
                ret_names.append(name.replace("ö", "o"))
            count = name.count("ü")
            if count > 0:
                ret_names.append(name.replace("ü", "ue"))
                ret_names.append(name.replace("ü", "u"))
            count = name.count("ß")
            if count > 0:
                ret_names.append(name.replace("ß", "ss"))
                ret_names.append(name.replace("ß", "s"))
        return ret_names


    # split at "-"
    ext_first_name = []
    for name in extending_german_letters(first_name): ext_first_name = ext_first_name + name.split("-")
    #print(ext_first_name)

    ext_last_name = []
    for name in extending_german_letters(last_name): ext_last_name = ext_last_name + name.split("-")
    #print(ext_last_name)


    return ext_first_name, ext_last_name

In [42]:
def find_name(f_names, l_names):
    if f_names and l_names:
        cypher_query_template = '''
        match (o:Officer)
        where tolower(o.name) contains '{f_name}' and  tolower(o.name) contains '{l_name}'
        return distinct o.name as name
        '''
    elif f_names:
        cypher_query_template = '''
        match (o:Officer)
        where tolower(o.name) contains '{f_name}'
        return distinct o.name as name
        '''
    else:
        cypher_query_template = '''
        match (o:Officer)
        where tolower(o.name) contains '{l_name}'
        return distinct o.name as name
        '''
    
    
    res_df = pd.DataFrame({'name' : []})
    
    
    
    for f_name in f_names:
        for l_name in l_names:
            cypher_query = cypher_query_template.format(f_name=f_name, l_name=l_name)
            df = gds.run_cypher(cypher_query)
            #print(df)
            res_df = pd.concat([res_df, df], ignore_index=True)
    
    return res_df

In [43]:
find_name(["wolfgang"], ["egger"])

,name
0,WOLFGANG PETER EGGER


In [47]:
df = df.dropna(subset=['Name'])
res = pd.DataFrame({'name' : [], 'count' : [], 'found_names' : []})

for i in range(0,df.shape[0]):
#for i in range(0,df.shape[0]):
        name = df.iloc[i,0]
        #print(name)
        f_names, l_names = names_lists(name)
        names = find_name(f_names, l_names)["name"]
        #print(names.head())
        res.loc[len(res.index)] = [df.iloc[i,0], len(names), list(names)]

print(res.head())

C:\Users\danie\anaconda3\envs\Challenge\lib\site-packages\pandas\core\dtypes\cast.py:948: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


                name  count found_names
0      Alfred Sauter      0          []
1  Günther Felbinger      0          []
2  Charles Smethurst      0          []
3         Ünal Bakir      0          []
4         Anton Kopp      0          []


In [48]:
res.to_csv("results_search_list.csv", index=False)
#gds.close()

The results can be looked up using the following cypher query:\
match (o)\
where o.name in [found_names]\
return distinct o